In [5]:
from Crypto.Util.number import getPrime, inverse, bytes_to_long
from string import ascii_letters, digits
from random import choice

pride = "".join(choice(ascii_letters + digits) for _ in range(16))
gluttony = getPrime(128)
greed = getPrime(128)
print((gluttony - 1) * (greed - 1))
lust = gluttony * greed
sloth = 65537
envy = inverse(sloth, (gluttony - 1) * (greed - 1))

anger = pow(bytes_to_long(pride.encode()), sloth, lust)

print(f"{anger = }")
print(f"{envy = }")

print("vainglory?")
vainglory = input("> ").strip()

if vainglory == pride:
    print("Conquered!")
    with open("/challenge/flag.txt") as f:
        print(f.read())
else:
    print("Hubris!")


78187947898741823723819901283859359942885026334130467448576246188178344330272
anger = 25461241846687380852721936396768879586247592208384371177352986818521376720319
envy = 72659422782200486599200504263101554825541705583526067092186690290959697647841
vainglory?
Hubris!


In [ ]:
from pwn import remote
from Crypto.Util.number import long_to_bytes

conn = remote('saturn.picoctf.net', 55778)

output = conn.recvuntil(b"vainglory?").decode()
print(output)  

sloth = 65537
lines = output.splitlines()
anger_line = lines[-3] 
anger = int(anger_line.split('= ')[1]) 
envy_line = lines[-2] 
envy = int(envy_line.split('= ')[1]) 

kphi = print(sloth * envy - 1)

# Gửi pride vào input
conn.sendline("5")
print(conn.recvall().decode())  # Nhận và in ra kết quả cuối cùng
conn.close()  # Đóng kết nối


[x] Opening connection to saturn.picoctf.net on port 55778
[x] Opening connection to saturn.picoctf.net on port 55778: Trying 13.59.203.175
[+] Opening connection to saturn.picoctf.net on port 55778: Done


In [ ]:
anger = 86416457058378576625326767439856513485474342688549161472382893060591648585291
envy = 79821998129035238600007925651785654893216818874375276952160456813342724618993
sloth = 65537
kphi = print(sloth * envy - 1)
print(87043762174113528683624864537368791898387137439945344315858085362514566458212)

5231294291382582432128719423441076464736750658569932525613739858176042143354944240
87043762174113528683624864537368791898387137439945344315858085362514566458212


In [ ]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, size, inverse
from math import gcd, prod
import operator
from functools import reduce
import itertools
from pwn import remote

conn = remote('saturn.picoctf.net', 52357)

output = conn.recvuntil(b"vainglory?").decode()
print(output)  

e = 65537
lines = output.splitlines()
anger_line = lines[-3] 
ct = int(anger_line.split('= ')[1]) 
envy_line = lines[-2] 
d = int(envy_line.split('= ')[1])

# encrypt(m) = m**e mod n = c
# d = e**-1 mod φ(n)
# decrypt(c) = c**d mod n = m

kφ = e * d - 1

def parse_pow(x):
    r = x.split('^')
    if len(r) == 1:
        return [int(r[0])]
    else:
        return [int(r[0])] * int(r[1])

print(kφ)

print('insert factors (use https://www.alpertron.com.ar/ECM.HTM) -- make sure to replace powers with "^":')
factors_in = input()

factors = sum([parse_pow(x.strip().replace(' ', '')) for x in factors_in.split('×')], [])
print(factors)

# remove some low hanging fruit
pfac = factors[-1] # biggest prime factor goes to p
qfac = factors[-2] # second biggest prime factor goes to q
factors = factors[:-2] # remove the top two factors since we've already assigned them
factors.remove(2) # used in p - 1 (since it's even)
factors.remove(2) # used by q - 1 (since it's even)
print(factors)

print('computing factors', len(factors))
T127 = 2**127
T129 = 2**129
ITERS = 3 ** len(factors)
steps = 0
pts = set()
for c in itertools.product([0,1,2], repeat=len(factors)):
    steps += 1
    h     =            prod(factors[i] for i, x in enumerate(c) if x == 0)
    psub1 = 2 * pfac * prod(factors[i] for i, x in enumerate(c) if x == 1)
    qsub1 = 2 * qfac * prod(factors[i] for i, x in enumerate(c) if x == 2)
    if h < e and T127 < psub1 < T129 and T127 < qsub1 < T129 and max(psub1+1, qsub1+1) < 2 * min(psub1+1, qsub1+1):
        n = (psub1 + 1) * (qsub1 + 1)
        pt = long_to_bytes(pow(ct, d, n))
        # there's no deduplication of the factors so we end up with multiple ways of computing p and q
        if len(pt) == 16:
            pts.add(pt)
            print(pt)
    if steps % 10000 == 0:
        print(steps, '/', ITERS, steps / ITERS)

print(pts)
pts_str = ' '.join(pt.decode('utf-8') for pt in pts)
conn.sendline(pts_str)
print(conn.recvall().decode()) 
conn.close()

[x] Opening connection to saturn.picoctf.net on port 52357


[x] Opening connection to saturn.picoctf.net on port 52357: Trying 13.59.203.175
[+] Opening connection to saturn.picoctf.net on port 52357: Done
anger = 27342589194274352624750171130686549371121497826400483263605927110769460636498
envy = 83137522450718146477012144332281982622211184143678362778283061364985166656545
vainglory?
5448583808852715165663944903104764295111854375224248861400336992677032867169989664
insert factors (use https://www.alpertron.com.ar/ECM.HTM) -- make sure to replace powers with "^":
[2, 2, 2, 2, 2, 3, 3, 17, 19, 23, 821, 2411, 35809, 5554207, 123314000270659046383, 52455832147628527555036392040053602243]
[2, 2, 2, 3, 3, 17, 19, 23, 821, 2411, 35809, 5554207]
computing factors 12
10000 / 531441 0.018816764231589206
20000 / 531441 0.03763352846317841
30000 / 531441 0.056450292694767625
40000 / 531441 0.07526705692635682
50000 / 531441 0.09408382115794604
b'QytfGDfTAGCQnlYr'
b'QytfGDfTAGCQnlYr'
60000 / 531441 0.11290058538953525
70000 / 531441 0.13171734962112444
800

C:\Users\Admin\AppData\Local\Temp\ipykernel_28828\1008169514.py:72: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(pts_str)


[x] Receiving all data: 76B
[+] Receiving all data: Done (76B)
[*] Closed connection to saturn.picoctf.net port 52357

> QytfGDfTAGCQnlYr
Conquered!
picoCTF{7h053_51n5_4r3_n0_m0r3_b2f9b414}

